In [81]:
from tensorflow import keras
import tensorflow as tf
import os
import time
import numpy as np
import pickle
from sklearn.model_selection import RandomizedSearchCV
import scikeras
from scikeras.wrappers import KerasRegressor

In [22]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

In [23]:
es = tf.keras.callbacks.EarlyStopping(patience=10,min_delta=1.0,verbose=1)

In [29]:
def get_callback(filename,name,value):
    root_logdir = os.path.join(os.curdir, filename)
    ts = int(time.time())
    filen = str(ts)+'_'+str(name)+'_'+str(value)
    return tf.keras.callbacks.TensorBoard(os.path.join(root_logdir, filen))

In [87]:
def build_model(n_hidden, n_neurons, optimizer, learning_rate, momentum=0):
    model = tf.keras.models.Sequential()
    model.add(keras.layers.Input(shape=13))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons))
    if optimizer == 'sgd': 
        opt = keras.optimizers.SGD(lr=learning_rate,nesterov=False)
    elif optimizer == 'nesterov':
        opt = keras.optimizers.SGD(lr=learning_rate,nesterov=True)
    elif optimizer == 'momentum':
        opt = keras.optimizers.SGD(lr=learning_rate,nesterov=False,momentum=momentum)
    elif optimizer == 'adam':
        opt = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mse',optimizer=opt,metrics=['mae'])
    return model

In [70]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [71]:
learning = [0.000001,0.00001,0.0001]
lr = []
for rate in learning:
    model = build_model(1,25,'sgd',rate)
    history = model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=[get_callback('tb_logs','lr',rate),es],verbose=0)
    result = (rate, history.history['loss'][-1], history.history['mae'][-1])
    lr.append(result)
lr

Epoch 11: early stopping


[(1e-06, 146.8929443359375, 9.387727737426758),
 (1e-05, 91.55538177490234, 7.155919075012207),
 (0.0001, inf, 3.974670369937499e+24)]

In [72]:
hidden = [0,1,2,3]
hl = []
for hid in hidden:
    model = build_model(hid,25,'sgd',1e-05)
    history = model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=[get_callback('tb_logs','hl',hid),es],verbose=0)
    result = (hid, history.history['loss'][-1], history.history['mae'][-1])
    hl.append(result)
hl

Epoch 11: early stopping
Epoch 89: early stopping
Epoch 76: early stopping


[(0, 23263.36328125, 72.36067199707031),
 (1, 96.52108764648438, 7.339536666870117),
 (2, 82.41569519042969, 6.707451820373535),
 (3, 78.71813201904297, 6.617793560028076)]

In [73]:
neurons = [5,25,125]
nn = []
for neuron in neurons:
    model = build_model(1,neuron,'sgd',1e-05)
    history = model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=[get_callback('tb_logs','nn',neuron),es],verbose=0)
    result = (neuron, history.history['loss'][-1], history.history['mae'][-1])
    nn.append(result)
nn

Epoch 78: early stopping
Epoch 88: early stopping


[(5, 86.2364730834961, 6.978971481323242),
 (25, 96.8493423461914, 7.392211437225342),
 (125, 95.11186218261719, 7.335632801055908)]

In [74]:
opti = ['sgd','nesterov','momentum','adam']
opt = []
for optimizer in opti:
    model = build_model(1,25,optimizer,1e-05,0.5)
    history = model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=[get_callback('tb_logs','opt',optimizer),es],verbose=0)
    result = (optimizer, history.history['loss'][-1], history.history['mae'][-1])
    opt.append(result)
opt

[('sgd', 96.74095153808594, 7.328789234161377),
 ('nesterov', 95.3700180053711, 7.234646320343018),
 ('momentum', 81.38885498046875, 6.604732513427734),
 ('adam', 21956.6328125, 123.32284545898438)]

In [75]:
moment = [0.1,0.5,0.9]
mom = []
for momentum in moment:
    model = build_model(1,25,'momentum',1e-05,momentum)
    history = model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=[get_callback('tb_logs','mom',momentum),es],verbose=0)
    result = (momentum, history.history['loss'][-1], history.history['mae'][-1])
    mom.append(result)
mom

Epoch 58: early stopping


[(0.1, 95.25955200195312, 7.269810676574707),
 (0.5, 83.32598876953125, 6.687946796417236),
 (0.9, 72.64730072021484, 6.113651752471924)]

In [90]:
param_distribs = {
"model__n_hidden": [0,1,2,3],
"model__n_neurons": [5,25,125],
"model__learning_rate": [0.000001,0.00001,0.0001],
"model__optimizer": ['sgd','nesterov', 'momentum','adam'],
"model__momentum": [0.1,0.5,0.9]
}

In [91]:
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [92]:
rnd_search_cv = RandomizedSearchCV(keras_reg,param_distribs,n_iter=20,cv=3,verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)

INFO:tensorflow:Assets written to: C:\Users\Pawel\AppData\Local\Temp\tmp5zswvbj1\assets
Fitting 3 folds for each of 20 candidates, totalling 60 fits
INFO:tensorflow:Assets written to: C:\Users\Pawel\AppData\Local\Temp\tmp8vndrmoq\assets
Epoch 1/100


c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


8/8 [==============================] - 0s 20ms/step - loss: 23399.2207 - mae: 72.8680 - val_loss: 24142.5352 - val_mae: 71.7670
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 23399.2207 - mae: 72.8679 - val_loss: 24142.5352 - val_mae: 71.7670
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 23399.2207 - mae: 72.8680 - val_loss: 24142.5352 - val_mae: 71.7670
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 23399.2227 - mae: 72.8680 - val_loss: 24142.5352 - val_mae: 71.7670
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: 23399.2227 - mae: 72.8680 - val_loss: 24142.5352 - val_mae: 71.7670
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 23399.2207 - mae: 72.8680 - val_loss: 24142.5352 - val_mae: 71.7670
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 23399.2207 - mae: 72.8680 - val_loss: 24142.5352 - val_mae: 71.7670
Epoch 8/100
8/8 [=====================

c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_tr

INFO:tensorflow:Assets written to: C:\Users\Pawel\AppData\Local\Temp\tmpw7d3jdns\assets


...

c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_tr

INFO:tensorflow:Assets written to: C:\Users\Pawel\AppData\Local\Temp\tmpxnrjza8y\assets
INFO:tensorflow:Assets written to: C:\Users\Pawel\AppData\Local\Temp\tmpocfnkz_7\assets
Epoch 1/100
 1/12 [=>............................] - ETA: 1s - loss: 25907.6699 - mae: 71.7324

c:\users\pawel\appdata\local\programs\python\python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


12/12 [==============================] - 0s 8ms/step - loss: 23378.0195 - mae: 72.6294 - val_loss: 22248.2461 - val_mae: 69.9812
Epoch 2/100
12/12 [==============================] - 0s 3ms/step - loss: 23378.0195 - mae: 72.6294 - val_loss: 22248.2461 - val_mae: 69.9812
Epoch 3/100
12/12 [==============================] - 0s 3ms/step - loss: 23378.0195 - mae: 72.6294 - val_loss: 22248.2461 - val_mae: 69.9812
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 23378.0215 - mae: 72.6294 - val_loss: 22248.2461 - val_mae: 69.9812
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: 23378.0195 - mae: 72.6294 - val_loss: 22248.2461 - val_mae: 69.9812
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 23378.0195 - mae: 72.6294 - val_loss: 22248.2461 - val_mae: 69.9812
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 23378.0215 - mae: 72.6294 - val_loss: 22248.2461 - val_mae: 69.9812
Epoch 8/100
12/12 [======

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x000001D39B44D580>], model=<function build_model at 0x000001D3A2FF59D0>),
                   n_iter=20,
                   param_distributions={'model__learning_rate': [1e-06, 1e-05,
                                                                 0.0001],
                                        'model__momentum': [0.1, 0.5, 0.9],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [5, 25, 125],
                                        'model__optimizer': ['sgd', 'nesterov',
                                                             'momentum',
                                                             'adam']},
                   verbose=2)

In [94]:
rnd_search_cv.best_params_

{'model__optimizer': 'adam',
 'model__n_neurons': 125,
 'model__n_hidden': 0,
 'model__momentum': 0.9,
 'model__learning_rate': 1e-06}